In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random

# makes the random numbers predictable so that same set of numbers will appear every time
from pygments.lexers import go
import plotly.offline as pyoff
import plotly.graph_objs as go

np.random.seed(42)
random.seed(42)
warnings.filterwarnings('ignore')

# Make the default figures a bit bigger
plt.rcParams['figure.figsize'] = (7,4.5)
plt.rcParams["figure.dpi"] = 140

# Plotting parameters
sns.set(style="ticks")
sns.set_context("poster", font_scale = .5, rc={"grid.linewidth": 5})

# Read the data
df1 = pd.read_csv('./data/olist_orders_dataset.csv')
df2 = pd.read_csv('./data/olist_customers_dataset.csv')
df3 = pd.read_csv('./data/olist_order_payments_dataset.csv')
df4 = pd.read_csv('./data/olist_order_items_dataset.csv')

#New Customer Ratio
#create a dataframe contaning CustomerID and first purchase date
cols = ['customer_id', 'customer_unique_id']
customers = df2[cols]
customers = customers.set_index('customer_id')

# Get only the required columns
cols = ['order_id', 'customer_id', 'order_purchase_timestamp', 'order_status']
orders = df1[cols]
orders = orders.set_index('customer_id')

customers_orders = pd.merge(orders, customers, on='customer_id')

# Add MinPurchaseDate, MinPurchaseYearMonth, MinPurchaseYear columns
cust_min_purchase = customers_orders.groupby('customer_unique_id').order_purchase_timestamp.min().reset_index()
cust_min_purchase.columns = ['customer_unique_id', 'MinPurchaseDate']
# converting the type of order_purchase_timestamp Field from string to datetime.
cust_min_purchase['MinPurchaseDate'] = pd.to_datetime(cust_min_purchase['MinPurchaseDate'])
cust_min_purchase['MinPurchaseYearMonth'] = cust_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)
cust_min_purchase['MinPurchaseYear'] = cust_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year)

# merge MinPurchaseDate columns with customers table for comparison
cust_orders_minpurchasetime = pd.merge(customers_orders, cust_min_purchase, on='customer_unique_id')

# add UserType column
cust_orders_minpurchasetime['UserType'] = 'New'
# Add PurchaseTimeStamp, PurchaseYearMonth, PurchaseYear columns
cust_orders_minpurchasetime['PurchaseTimeStamp'] = pd.to_datetime(cust_orders_minpurchasetime['order_purchase_timestamp'])
cust_orders_minpurchasetime['PurchaseYearMonth'] = cust_orders_minpurchasetime['PurchaseTimeStamp'].map(lambda date: 100*date.year + date.month)
cust_orders_minpurchasetime['PurchaseYear'] = cust_orders_minpurchasetime['PurchaseTimeStamp'].map(lambda date: date.year)

# New customer is whoever did first purchase in the time window (monthly)
cust_orders_minpurchasetime.loc[cust_orders_minpurchasetime['PurchaseYearMonth'] > cust_orders_minpurchasetime['MinPurchaseYearMonth'], 'UserType'] = 'Existing'

# Get only the required columns
cols = ['order_id', 'price']
order_items = df4[cols]
# we need even duplicates to calculate the price
order_items = order_items.set_index('order_id')

# Combine the prices of items within same order
order_items = order_items.groupby(['order_id'])['price'].sum().reset_index()

orders_cust_usertype_price = pd.merge(cust_orders_minpurchasetime, order_items, on='order_id')

# Groupby unique customer ids

#calculate the Revenue per month for each user type
orders_cust_revenue = orders_cust_usertype_price.groupby(['PurchaseYearMonth','UserType'])['price'].sum().reset_index()

a = 'test'
#filtering the dates and plot the result
# orders_cust_revenue = orders_cust_revenue.query("PurchaseYearMonth != 201012 and PurchaseYearMonth != 201112")

plot_data = [
    go.Scatter(
        x=orders_cust_revenue.query("UserType == 'Existing'")['PurchaseYearMonth'],
        y=orders_cust_revenue.query("UserType == 'Existing'")['price'],
        name = 'Existing'
    ),
    go.Scatter(
        x=orders_cust_revenue.query("UserType == 'New'")['PurchaseYearMonth'],
        y=orders_cust_revenue.query("UserType == 'New'")['price'],
        name = 'New'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category", "categoryorder": "category ascending", 'title': "Period"},
        title='(Monthly) New vs Existing Customers Revenue',
        yaxis= {'title': "Revenue"},
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

################################################
# New Customer Ratio

#create a dataframe that shows new user ratio - we also need to drop NA values (first month new user ratio is 0)
user_ratio = cust_orders_minpurchasetime.query("UserType == 'New'").groupby(['PurchaseYearMonth'])['customer_unique_id'].nunique()/cust_orders_minpurchasetime.query("UserType == 'Existing'").groupby(['PurchaseYearMonth'])['customer_unique_id'].nunique()
user_ratio = user_ratio.reset_index()
user_ratio = user_ratio.dropna()
a = 'test'

#plot the result
plot_data = [
    go.Bar(
        x=user_ratio['PurchaseYearMonth'],
        y=user_ratio['customer_unique_id'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New Customer Ratio'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
# pyoff.iplot(fig)
